In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_date, coalesce, trim,initcap

#silver

##reading from bronze

In [0]:
df_bronze = spark.sql("SELECT * FROM main_fmcg.bronze.products")
df_bronze.show(10)

##Remove dublicates

In [0]:

# Count before
before_count = df_bronze.count()
print(f"Total rows BEFORE removing duplicates: {before_count}")

# Remove duplicates
df_bronze = df_bronze.dropDuplicates()

# Count after
after_count = df_bronze.count()
print(f"Total rows AFTER removing duplicates: {after_count}")

print(f"Number of duplicates removed: {before_count - after_count}")

##First Letter Capital in category

In [0]:
#df_bronze.select('category').distinct().show()
df_bronze = df_bronze.withColumn(
    "category",
    initcap(col("category"))
)
#df_bronze.select('category').distinct().show()

##Replace 'protien' → 'protein' in (product_name , category)

In [0]:

df_bronze = df_bronze.withColumn(
    "product_name",
    F.regexp_replace(F.col("product_name"), "(?i)protien", "Protein")
).withColumn(
    "category",
    F.regexp_replace(F.col("category"), "(?i)protien", "Protein")
)


##Standardizing Customer Attributes to Match Parent Company Data Model

In [0]:
### 1: Add division column

# Category → Division mapping
division_mapping = {
    "Energy Bars": "Nutrition Bars",
    "Protein Bars": "Nutrition Bars",
    "Granola & Cereals": "Breakfast Foods",
    "Recovery Dairy": "Dairy & Recovery",
    "Healthy Snacks": "Healthy Snacks",
    "Electrolyte Mix": "Hydration & Electrolytes"
}

df_bronze = df_bronze.withColumn(
    "division",
    F.coalesce(
        F.create_map([F.lit(x) for x in sum(division_mapping.items(), ())])
        .getItem(F.col("category")),
        F.lit("Other")
    )
)

### 2: Variant column
df_bronze = df_bronze.withColumn(
    "variant",
    F.regexp_extract(F.col("product_name"), r"\((.*?)\)", 1)
)

### 3: Create new column: product_code  

# Invalid product_ids are replaced with a fallback value to avoid losing fact records and ensure downstream joins remain consistent

df_bronze = (
    df_bronze
    # 1. Generate deterministic product_code from product_name
    .withColumn(
        "product_code",
        F.sha2(F.col("product_name").cast("string"), 256)
    )
    # 2. Clean product_id: keep only numeric IDs, else set to 999999
    .withColumn(
        "product_id",
        F.when(
            F.col("product_id").cast("string").rlike("^[0-9]+$"),
            F.col("product_id").cast("string")
        ).otherwise(F.lit(999999).cast("string"))
    )
    # 3. Rename product_name → product
    .withColumnRenamed("product_name", "product")
)

##structure saving

In [0]:
df_bronze = df_bronze.select("product_code", "division", "category", "product", "variant", "product_id", "read_timestamp", "file_name", "file_size")

In [0]:
display(df_bronze)

##write to silver

In [0]:
df_bronze.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true")\
 .option("overwriteSchema", "true") \
 .mode("overwrite") \
 .saveAsTable("main_fmcg.silver.products")

In [0]:
#display(df_bronze)
#display(df_bronze.limit(10))